In [ ]:
# ('trip_id',	'taxi_id',	'trip_start_timestamp',	'trip_end_timestamp')
taxi_data = [
    ('85','taxi_001','07/19/2016 02:45:00 PM','07/19/2016 02:48:00 PM'),
    ('22','taxi_002','01/22/2015 04:15:00 PM','01/22/2015 04:30:00 PM'),
    ('86','taxi_001','07/19/2016 03:00:00 PM','07/19/2016 03:15:00 PM'),
    ('23','taxi_002','01/22/2015 04:45:00 PM','01/22/2015 05:30:00 PM'),
    ('87','taxi_001','07/19/2016 03:25:00 PM','07/19/2016 03:45:00 PM'),
    ('24','taxi_002','01/22/2015 05:45:00 PM','01/22/2015 06:30:00 PM'),
    ('88','taxi_001','07/19/2016 03:55:00 PM','07/19/2016 04:05:00 PM'),
    ('25','taxi_002','01/22/2015 06:45:00 PM','01/22/2015 07:10:00 PM'),
    ('89','taxi_001','07/19/2016 04:25:00 PM','07/19/2016 04:45:00 PM'),
    ('26','taxi_002','01/22/2015 07:15:00 PM','01/22/2015 08:30:00 PM'),
    ('90','taxi_001','07/19/2016 04:50:00 PM','07/19/2016 05:05:00 PM'),
    ('27','taxi_002','01/22/2015 08:35:00 PM','01/22/2015 08:50:00 PM'),
    ('91','taxi_001','07/19/2016 05:15:00 PM','07/19/2016 06:45:00 PM'),
    ('28','taxi_002','01/22/2015 09:15:00 PM','01/22/2015 10:30:00 PM')
]

In [ ]:
import os
import re
import time
import uuid
from pyspark import SparkContext, SparkConf


conf = SparkConf()
ctx = SparkContext(master='local[*]', appName="spark-secondary-sort", conf=conf)
ctx.setLogLevel('ERROR')




25/10/08 09:55:52 WARN Utils: Your hostname, vmware-ubuntu-24.04 resolves to a loopback address: 127.0.1.1; using 192.168.154.133 instead (on interface ens33)
25/10/08 09:55:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 09:55:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rawRDD = ctx.parallelize(taxi_data)
print(rawRDD.collect())

[('b258df504d2fbf85', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 02:45:00 PM', '07/19/2016 02:48:00 PM'), ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 04:15:00 PM', '01/22/2015 04:30:00 PM'), ('b258df504d2fbf86', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:00:00 PM', '07/19/2016 03:15:00 PM'), ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 04:45:00 PM', '01/22/2015 05:30:00 PM'), ('b258df504d2fbf87', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:25:00 PM', '07/19/2016 03:45:00 PM'), ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 05:45:00 PM', '01/22/2015 06:30:00 PM'), ('b258df504d2fbf88', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:55:00 PM', '07/19/2016 04:05:00 PM'), ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 06:45:00 PM', '01/22/2015 07:10:00 PM'), ('b258df504d2fbf89', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 04:25:00 PM', '07/19/2016 04:45:00 PM'), ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 07:15:00 PM', '01/22/2015

In [4]:
# (taxi_id -> ('trip_id',	'taxi_id',	'trip_start_timestamp',	'trip_end_timestamp'))
def make_pair(entry):
    key = entry[1]
    return key, entry

pairRDD = rawRDD.map(make_pair)
print(pairRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', ('b258df504d2fbf85', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 02:45:00 PM', '07/19/2016 02:48:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 04:15:00 PM', '01/22/2015 04:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('b258df504d2fbf86', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:00:00 PM', '07/19/2016 03:15:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 04:45:00 PM', '01/22/2015 05:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('b258df504d2fbf87', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:25:00 PM', '07/19/2016 03:45:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '01/22/2015 05:45:00 PM', '01/22/2015 06:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('b258df504d2fbf88', 'b7b0be6d3ec6c589aeac84c0', '07/19/2016 03:55:00 PM', '07/19/2016 04:05:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('b258df6e5ace922', '0d61c4f9c8cb8d280fce3887', '

In [ ]:
# (taxi_id -> ('trip_start_timestamp',	'trip_end_timestamp'))
compressedRDD = pairRDD.mapValues(lambda x: (x[2], x[3]))
print(compressedRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 02:45:00 PM', '07/19/2016 02:48:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 04:15:00 PM', '01/22/2015 04:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 03:00:00 PM', '07/19/2016 03:15:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 04:45:00 PM', '01/22/2015 05:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 03:25:00 PM', '07/19/2016 03:45:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 05:45:00 PM', '01/22/2015 06:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 03:55:00 PM', '07/19/2016 04:05:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 06:45:00 PM', '01/22/2015 07:10:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 04:25:00 PM', '07/19/2016 04:45:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 07:15:00 PM', '01/22/2015 08:30:00 PM')), ('b7b0be6d3ec6c589aeac84c0', ('07/19/2016 04:50:00 PM', '07/19/2016 05:05:00 PM')), ('0d61c4f9c8cb8d280fce3887', ('01/22/2015 08:35:00 PM', '01/22/2015 08:50:0

In [ ]:
# (taxi_id -> [('trip_start_timestamp',	'trip_end_timestamp'), ('trip_start_timestamp',	'trip_end_timestamp'), ...])
groupedRDD = compressedRDD.groupByKey(numPartitions=2)
print(groupedRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', <pyspark.resultiterable.ResultIterable object at 0x7aec6e390a50>), ('0d61c4f9c8cb8d280fce3887', <pyspark.resultiterable.ResultIterable object at 0x7aec6e390890>)]


In [ ]:
#
# Sort the group based on trip_start_timestamp
#
def sort_group(group):
    return sorted(group, key=lambda entry: entry[0], reverse=False)

sortedRDD = groupedRDD.mapValues(sort_group)
print(sortedRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', [('07/19/2016 02:45:00 PM', '07/19/2016 02:48:00 PM'), ('07/19/2016 03:00:00 PM', '07/19/2016 03:15:00 PM'), ('07/19/2016 03:25:00 PM', '07/19/2016 03:45:00 PM'), ('07/19/2016 03:55:00 PM', '07/19/2016 04:05:00 PM'), ('07/19/2016 04:25:00 PM', '07/19/2016 04:45:00 PM'), ('07/19/2016 04:50:00 PM', '07/19/2016 05:05:00 PM'), ('07/19/2016 05:15:00 PM', '07/19/2016 06:45:00 PM')]), ('0d61c4f9c8cb8d280fce3887', [('01/22/2015 04:15:00 PM', '01/22/2015 04:30:00 PM'), ('01/22/2015 04:45:00 PM', '01/22/2015 05:30:00 PM'), ('01/22/2015 05:45:00 PM', '01/22/2015 06:30:00 PM'), ('01/22/2015 06:45:00 PM', '01/22/2015 07:10:00 PM'), ('01/22/2015 07:15:00 PM', '01/22/2015 08:30:00 PM'), ('01/22/2015 08:35:00 PM', '01/22/2015 08:50:00 PM'), ('01/22/2015 09:15:00 PM', '01/22/2015 10:30:00 PM')])]


In [15]:
from datetime import datetime

def calculate_loss(entry):
    group = entry
    loss = 0
    _, prev_end = group[0]

    for item in group:
        start, end = item
        delta = datetime.strptime(start, '%m/%d/%Y %I:%M:%S %p').timestamp()\
                - datetime.strptime(prev_end, '%m/%d/%Y %I:%M:%S %p').timestamp()
        if delta > 0:
            loss += delta
        prev_end = end
        
    return loss


lossRDD = sortedRDD.mapValues(calculate_loss)

print(lossRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', 4020.0), ('0d61c4f9c8cb8d280fce3887', 4800.0)]


In [8]:
from datetime import datetime

def calculate_delta(entry):
    group = entry[0]
    loss = 0
    _, prev_end = entry[1][1]
    deltaArray = []
    for item in entry[1]:
        start, end = item
        delta = datetime.strptime(start, '%m/%d/%Y %I:%M:%S %p').timestamp()\
                - datetime.strptime(prev_end, '%m/%d/%Y %I:%M:%S %p').timestamp()
        if delta > 0:
            loss += delta
            deltaArray.append(delta)
        
        prev_end = end

    return (group, deltaArray)


deltaRDD = sortedRDD.map(calculate_delta)

print(deltaRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', [720.0, 600.0, 600.0, 1200.0, 300.0, 600.0]), ('0d61c4f9c8cb8d280fce3887', [900.0, 900.0, 900.0, 300.0, 300.0, 1500.0])]


In [9]:
from datetime import datetime

def compute_loss(entry):
    loss = 0
    for item in entry:
        if item > 0:
            loss += item
        
    return loss


lossRDD = deltaRDD.mapValues(compute_loss)

print(lossRDD.collect())

[('b7b0be6d3ec6c589aeac84c0', 4020.0), ('0d61c4f9c8cb8d280fce3887', 4800.0)]
